In [9]:
!pip3 install --upgrade --quiet pip 
!pip3 install -I --upgrade --quiet setuptools
!pip3 install --upgrade --quiet pymongo
!pip3 install --quiet lightgbm

In [1]:
!pip install --upgrade --quiet pip 
!pip install -I --upgrade --quiet setuptools
!pip install --upgrade --quiet pymongo
!pip install --quiet lightgbm

In [19]:
% matplotlib inline

In [ ]:
%load_ext autoreload
%autoreload 2

In [18]:
import warnings
warnings.filterwarnings('ignore')

In [22]:
import modelling.fetch as fetch
from modelling.models import *
from modelling.utils import get_articles
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import *
from pymongo import MongoClient
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import fbeta_score, make_scorer
import lightgbm
from modelling.clustering import get_unique_items

In [23]:
df = fetch.create_df(fetch.get_labelled_articles("209.177.92.45:80"))

In [24]:
collection = MongoClient("209.177.92.45:80")['newsfilter'].news
df = create_df(get_articles(collection, label=True))

In [100]:
def group_by_ctx(li):
    grouped = {}
    for i in li:
        try:
            nd.concat(grouped[i.context.device_id], i, dim=1)
        except KeyError:
            grouped[i.context.device_id] = i
    return grouped        

In [ ]:
from modelling.clustering import get_unique_items
from modelling.utils import preprocessor

def get_uniques(df):
    lookup = [
        ('ge', 0.1, 'title'),
        ('tw', 0.5, 'body'),
        ('fa', 0.2, 'title') 
    ]

    return [get_unique_items(df[df._id.str.contains(p)], i, k) 
            for p,i,k in lookup]


def prepare_df(df, preprocessor, nlp, out_key = 'body'):
    uniques = get_uniques(df)
    unique = pd.concat([uniques[i].assign(text = uniques[i][t[2]]) 
                        for i,t in enumerate(lookup)] )
    unique['text'] = unique.text.map(preprocessor)
    unique = unique[unique.text.str.len() > 8]
    return unique.rename({ 'text': out_key })

In [ ]:
unique = prepare_df(df, preprocessor)

In [ ]:
v = vectorizer(unique)
tfidf = create_tfidf(v, unique)
word_count = create_word_count(unique).astype('float32')

In [ ]:
from sklearn.model_selection import StratifiedKFold
cv = StratifiedKFold(5, shuffle = True)

def evaluate(model, tfidf, df, method = 'predict_proba'):
    ans = [1 if l == 'accepted' else 0 for l in df.label]
    probs = cross_val_predict(model, tfidf, ans, cv=cv, method=method)
    predictions = cross_val_predict(model, tfidf, ans, cv=cv, method='predict')

    probs = [p[1] for p in probs] 
                           
    print (average_precision_score(ans, probs), roc_auc_score(ans, probs, average = None))
    print (precision_score(ans, predictions), recall_score(ans, predictions), fbeta_score(ans, predictions, beta = 1.5))
    # precision, recall, _ = precision_recall_curve(ans, guess)
    fpr, tpr, _ = roc_curve(ans, probs)
    plt.step(fpr, tpr)
    # plt.step(precision, recall)
    return probs, ans, predictions

In [ ]:
scorer = make_scorer(fbeta_score, beta = 1.5, pos_label = 'accepted')
parameters = [{ }]
grid = GridSearchCV(lightgbm.LGBMClassifier(n_estimators = 100, reg_lambda = 0.6, reg_alpha = 0.2), parameters, scoring = scorer)
# p,a,pr = evaluate(model, tfidf, df)
grid.fit(word_count, unique.label)
pd.DataFrame(grid.cv_results_).sort_values('rank_test_score')

In [ ]:
model = lightgbm.LGBMClassifier(n_estimators = 100, reg_lambda = 0.6, reg_alpha = 0.2)
_ = evaluate(model, word_count, unique)

In [57]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
g, ans, preds = evaluate(model, word_count, unique)

0.892214160644 0.923859491144
0.890710382514 0.762870514821 0.798116760829


In [ ]:
scorer = make_scorer(fbeta_score, beta = 1.5, pos_label = 'accepted')
parameters = [{ }]
model = svm.LinearSVC(dual = False, class_weight = 'balanced', C = 4., tol = 10e-6, max_iter = 4000)
grid = GridSearchCV(model, parameters, scoring = scorer)
# p,a,pr = evaluate(model, tfidf, df)
grid.fit(word_count, unique.label)
pd.DataFrame(grid.cv_results_).sort_values('rank_test_score')

In [13]:
from sklearn import svm

model = svm.LinearSVC(dual = False,  tol = 10e-6, max_iter = 4000, class_weight = 'balanced')
g,a,p = evaluate(model, word_count, unique, 'decision_function')

NameError: name 'evaluate' is not defined

In [ ]:
model = svm.OneClassSVM()
g,a,p = evaluate(model, tfidf, unique, 'decision_function')

In [59]:
model = MultinomialNB(alpha = .001, class_prior = [.5,.5])
g, ans, preds = evaluate(model, word_count, unique)

0.85228915512 0.909988127466
0.706242350061 0.90015600624 0.830032090295


In [ ]:
model.fit(tfidf, df.label)

# model.feature_log_prob_
sorted(zip(model.feature_log_prob_[0], v.get_feature_names()))[-50:]